> 동시출현빈도 기반

- 두 개의 단어가 주어진 문맥에서 서로 얼마나 연관되어 있는지
- 대상어와 다른 단어들이 같은 문맥 내에서 동시에 출현한 횟수를 세는 방법
- 두 단어가 같은 문맥 내에서 함께 나타나는 빈도가 높을수록 강한 연관관계가 성립한다는 가정
- 한 문장에서 한 단어가 여러번 중복되어 나타날 경우 1회로 계산하거나 n회로 계산, 1회로 계산하는 것이 일반적인 방법

In [1]:
import glob
# 긍정리뷰 100개 로딩
pos_review=(glob.glob("c:/vscode/data/imdb/train/pos/*.txt"))[0:100]
lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except :
        continue
len(lines_pos)

100

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd

tokenizer = RegexpTokenizer('[\w]+') #알파벳, 숫자, _
stop_words = stopwords.words('english')

#동시출현 단어 계산
count = {} #동시출현 빈도가 저장될 dict
for line in lines_pos:
    words = line.lower()
    tokens = tokenizer.tokenize(words)

    #불용어 제거, 불용어에 br 추가
    stopped_tokens = [i for i in list(set(tokens)) 
                        if i not in stop_words +["br"]]
    
    #글자수가 1인 단어 제외
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    for i, a in enumerate(stopped_tokens2):
        for b in stopped_tokens2[i+1: ]:
            if a>b:
                count[b, a] = count.get((b, a),0) + 1
            else :
                count[a, b] = count.get((a, b),0) + 1

In [3]:
#딕셔너리로부터 데이터프레임 생성
df = pd.DataFrame.from_dict(count, orient='index')

#리스트 구성
list1=[]
for i in range(len(df)):
    list1.append([df.index[i][0], df.index[i][1], df[0][i]])

In [4]:
df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
df3 = df2.sort_values(by=['freq'],ascending=False)
df3_pos = df3.reset_index(drop=True)

#동시출현 단어 페어 빈도 상위 20개 출력
df3_pos.head(20)
#film과 story 총 41회 동시에 출현

,term1,term2,freq
0,movie,one,41
1,film,story,41
2,movie,story,35
3,film,movie,35
4,one,story,33
5,good,movie,32
6,film,one,31
7,movie,see,30
8,one,see,27
9,film,like,27


In [5]:
import glob
# 부정리뷰 100개 로딩
pos_review=(glob.glob("c:/vscode/data/imdb/train/neg/*.txt"))[0:100]
lines_neg=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except :
        continue
len(lines_neg)

100

In [6]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd

tokenizer = RegexpTokenizer('[\w]+') #알파벳, 숫자, _
stop_words = stopwords.words('english')

#동시출현 단어 계산
count = {} #동시출현 빈도가 저장될 dict
for line in lines_neg:
    words = line.lower()
    tokens = tokenizer.tokenize(words)

    #불용어 제거, 불용어에 br 추가
    stopped_tokens = [i for i in list(set(tokens)) 
                        if i not in stop_words +["br"]]
    
    #글자수가 1인 단어 제외
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    for i, a in enumerate(stopped_tokens2):
        for b in stopped_tokens2[i+1: ]:
            if a>b:
                count[b, a] = count.get((b, a),0) + 1
            else :
                count[a, b] = count.get((a, b),0) + 1

In [7]:
#딕셔너리로부터 데이터프레임 생성
df = pd.DataFrame.from_dict(count, orient='index')

#리스트 구성
list1=[]
for i in range(len(df)):
    list1.append([df.index[i][0], df.index[i][1], df[0][i]])

df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
df3 = df2.sort_values(by=['freq'],ascending=False)
df3_neg = df3.reset_index(drop=True)

#동시출현 단어 페어 빈도 상위 20개 출력
df3_neg.head(20)
#film과 movie가 총 42회 동시에 출현

,term1,term2,freq
0,film,movie,42
1,like,movie,40
2,movie,one,38
3,film,one,35
4,like,one,33
5,even,movie,32
6,good,movie,32
7,even,like,31
8,good,one,30
9,film,like,29


> 통계적 가중치 기반

- 통계적으로 가중치를 구한 후 두 단어 간의 유사도를 단어간의 연관도로 적용하는 방법
- 1.단어마다 가중치를 할당해야 함(출현빈도, tf-idf 등으로 계산)
- 2.단어간의 유사도 계산(cosine similarity 등의 방법)

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')

#TF-IDF 가중치 할당
vec = TfidfVectorizer(stop_words=stop_words)
vector_lines_pos = vec.fit_transform(lines_pos)
A=vector_lines_pos.toarray()
print(A.shape)
# print(A)
#x축 단어, y축 문서
#현재 상태는 100개의 문서의 유사도

(100, 3989)


In [10]:
# 단어간의 유사도를 구하는 것이 목적이므로
# 단어-문서 행렬로 변경
# x축 문서, y축 단어
A=A.transpose()
print(A.shape)

(3989, 100)


In [11]:
import numpy as np
from scipy import sparse

#밀집행렬(dense array)
a=np.array([[0.5,0,0],[0,1,0],[0.7,0,1.5]])

#밀집행렬을 희소행렬(sparse array)로 변환
#밀집행렬의 단점: 0이 많을 경우 메모리 낭비가 될 수 있음
#희소행렬은 0이 아닌 값들의 위치와 값만 기록하여 메모리를 절약하는 방식
b=sparse.csr_matrix(a)
print(b)
# (0,0) 0.5 => 인덱스 0,0에 값 0.5
# (1,1) 1 => 인덱스 1,1에 값 1
# (2,0) 0.7 => 인덱스 2,0에 값 0.7
# (2,2) 1 => 인덱스 2,2에 값 1.5
c=b.toarray() #희소행렬을 밀집행렬로 변환
print(c)

  (0, 0)	0.5
  (1, 1)	1.0
  (2, 0)	0.7
  (2, 2)	1.5
[[0.5 0.  0. ]
 [0.  1.  0. ]
 [0.7 0.  1.5]]


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

A_sparse = sparse.csr_matrix(A)

#코사인 유사도 계산
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)

# todok() 행렬을 딕셔너리 형태로 변환
list(similarities_sparse.todok().items())[35000:35010]
#list(similarities_sparse.todok().items())[-10:]
#단어 이름이 아닌 인덱스 형태로 출력됨
# (1469,108), 0.37 1469 단어와 108 단어의 유사도는 37%
vec.get_feature_names_out()[1469]

'friday'

In [13]:
#위의 결과값을 데이터프레임으로 출력
df=pd.DataFrame(list(similarities_sparse.todok().items()),columns=["words","weight"])
df2=df.sort_values(by=['weight'],ascending=False)
df2=df2.reset_index(drop=True)

#단어 자신끼리의 짝은 1이 되므로 1보다 작은 항목들만 출력시킴
df3=df2.loc[np.round(df2['weight']) < 1]
df3=df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(2613, 3250)",0.499980
1,"(3250, 2613)",0.499980
2,"(2545, 1188)",0.499979
3,"(1188, 2545)",0.499979
4,"(1767, 2543)",0.499965
5,"(2543, 1767)",0.499965
6,"(1074, 250)",0.499935
7,"(250, 1074)",0.499935
8,"(2322, 525)",0.499934
9,"(525, 2322)",0.499934


In [29]:
for i, row in enumerate(df3.iterrows()):
    a=vec.get_feature_names_out()[row[1][0][0]]
    b=vec.get_feature_names_out()[row[1][0][1]]
    print(f'{a},{b}=>{row[1][1]:.2f}')    
    if i>10:
        break

physically,society=>0.50
society,physically=>0.50
past,essence=>0.50
essence,past=>0.50
iceberg,passengers=>0.50
passengers,iceberg=>0.50
dying,art=>0.50
art,dying=>0.50
move,care=>0.50
care,move=>0.50
horner,long=>0.50
long,horner=>0.50


> word2vec (워드임베딩)

- 단어의 의미는 그 단어 주변 단어의 분포로 이해될 수 있다
- 단어의 의미는 단어 벡터 안에 인코딩될 수 있다.
- 단순히 출현횟수만을 고려하는 것이 아닌 단어 위치, 순서도 고려하는 방법
- CBOW 주변 단어로 중심 단어를 예측하는 방법
- Skip-gram 중심 단어로 주변 단어를 예측하는 방법(최근에 더 많이 사용되는 방법)

In [30]:
from nltk.corpus import stopwords
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import RegexpTokenizer

stop_words = stopwords.words('english')
tokenizer = RegexpTokenizer('[\w]+')

#단어 추출
text=[]
for line in lines_pos:
    words = line.lower()
    tokens = tokenizer.tokenize(words)
    stopped_tokens = [i for i in list(set(tokens)) 
                        if i not in stop_words+["br"]]
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    text.append(stopped_tokens2)
    
# word2vec 모형 생성 , sg=1 skip-gram을 적용, 
# window=2 중심 단어로부터 좌우 2개의 단어까지 학습에 적용
# min_count=3 최소 3회 이상 출현한 단어들을 대상으로 학습
model = Word2Vec(text, vector_size=10, sg=1, window=2, min_count=3)

# 두 단어의 유사도 계산
model.wv.similarity('film', 'movie')

0.9413078

In [39]:
#good과 가장 유사한 단어 5개
model.wv.most_similar("good",topn=5)

[('directed', 0.892633855342865),
 ('near', 0.8925931453704834),
 ('needed', 0.886659562587738),
 ('like', 0.8846356272697449),
 ('school', 0.8817977905273438)]

In [32]:
#모델에 저장된 단어의 갯수
len(model.wv.index_to_key)

895

In [33]:
#모델에 저장된 단어 텍스트
model.wv.index_to_key[0]

'movie'

In [34]:
#단어에 해당하는 벡터값
model.wv.vectors[0]

array([ 0.16575187, -0.13935485,  0.30233198,  0.09304485,  0.08483201,
       -0.02816552,  0.24418698,  0.23198113, -0.23904102, -0.25841346],
      dtype=float32)